# Converting cubed sphere to lat/lon grid using JEDI

## prerequisite 
`jedi-bundle` is already built in your `JEDI_BUILD`.

## Usage and limitations
`fv3jedi_converttolatlon.x` can be used to convert a cubed sphere file to a lat/lon grid. However, the user needs to specify which model level they want to perform the conversion on. This means that the executable doesn't automatically regrid all the model levels.

An example of an input YAML file is below.
The user needs to specify:
- Input geometry (under `state geometry`)
- Details about the input file (under `state`)
- Details about the `latlon interpolation`. More info here: ???? 

```yaml
states to latlon:
- state geometry:
    fms initialization:
      namelist filename: ./fmsmpp.nml
      field table filename: ./field_table_gmao
    akbk: ./akbk72.nc4
    npx: 13
    npy: 13
    npz: 72
    field metadata override: ./geos_cf.yaml

  state:
    datetime: 2020-09-03T15:00:00Z
    filetype: cube sphere history
    state variables: [DELP,NO2,NO,O3,PS,SPHU]
    datapath: ./
    filename: geos_cf.bkg.20200903_150000z.nc4

  latlon interpolation:
    local interpolator type: oops unstructured grid interpolator
    resolution in degrees: 5.0  # low resolution for testing
    # need to use long names (see "field metadata override" file)
    variables to output: [air_pressure_thickness,volume_mixing_ratio_of_no2] 
    pressure levels in hPa: [925, 850, 700, 500, 250]
    model levels: [71, 10]
    bottom model level: true
    datapath: ./
    prefix: latlon.geos
```

First, set the path to your `JEDI_BUILD` and `MPIEXEC`. For example for me on Discover, it looks like this:

```bash
export MPIEXEC=/discover/swdev/jcsda/spack-stack/openmpi-4.1.3/gcc-10.1.0/bin/mpiexec
export JEDI_BUILD=/discover/nobackup/mabdiosk/jedi-bundle/build-gnu-spack-1.5.1
```

All the files that you need are under `convert_to_latlon` directory. So `cd convert_to_latlon` and then you can run the application this command: 

```bash
$MPIEXEC "-n" "6" $JEDI_BUILD/bin/fv3jedi_converttolatlon.x ./convert_to_latlon/converttolatlon_geos.yaml
```

In this example, two files will be generated. The "PressureLevels" file will include the `variables to output` values in latlon grid for levels specified under `pressure levels in hPa`. Similarly, the "modelLevels" file will include the `variables to output` values in latlon grid for levels specified under `model levels`.

Another example of this application is ctest `fv3jedi_test_tier1_converttolatlon_gfs` in [fv3-jedi](https://github.com/JCSDA-internal/fv3-jedi/blob/1.8.0/test/CMakeLists.txt#L992). In this [example](https://github.com/JCSDA-internal/fv3-jedi/blob/1.8.0/test/testinput/converttolatlon_gfs.yaml), filetype of `fms restart` is converted from cubed sphere to latlon grid. 